# An experiment template
> Modularity might not be the solution, but it's all we got.

In [ ]:
# default_exp experiment

In [ ]:
# hide
import blackhc.project.script

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
from dataclasses import dataclass
from typing import Optional

import torch
import torch.utils.data
from ignite.contrib.engines.common import (
    add_early_stopping_by_val_score,
    setup_common_training_handlers,
)
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Accuracy, Loss, RunningAverage
from torch import nn
from torch.utils.data import Dataset

from batchbald_redux.active_learning import (
    ActiveLearningData,
    RandomFixedLengthSampler,
    get_base_indices,
)
from batchbald_redux.batchbald import get_bald_batch
from batchbald_redux.black_box_model_training import evaluate, get_predictions, train
from batchbald_redux.consistent_mc_dropout import (
    GeometricMeanPrediction,
    SamplerModel,
    geometric_mean_loss,
    multi_sample_loss,
)
from batchbald_redux.example_models import BayesianMNISTCNN
from batchbald_redux.fast_mnist import FastMNIST
from batchbald_redux.repeated_mnist import create_repeated_MNIST_dataset

In [ ]:
# exports


@dataclass
class Experiment:
    acquisition_size: int = 10
    max_training_set: int = 300
    num_pool_samples: int = 20
    num_eval_samples: int = 4
    num_training_samples: int = 1
    num_patience_epochs: int = 3
    max_training_epochs: int = 10
    device = "cuda"
    validation_set_size: int = 1024
    initial_set_size: int = 20
    samples_per_epoch: int = 32768

    def load_dataset(self) -> (ActiveLearningData, Dataset, Dataset):
        train_dataset, test_dataset = create_repeated_MNIST_dataset(num_repetitions=1, add_noise=False)
        active_learning_data = ActiveLearningData(train_dataset)

        validation_dataset = active_learning_data.extract_dataset_from_pool(self.validation_set_size)

        return active_learning_data, validation_dataset, test_dataset

    def new_model(self):
        return BayesianMNISTCNN()

    def new_optimizer(self, model):
        return torch.optim.Adam(model.parameters(), weight_decay=5e-4)

    def get_candidate_batch(self, model, pool_loader):
        # Evaluate pool set
        bald_model = SamplerModel(model, self.num_pool_samples)
        pool_log_probs_N_K_C = get_predictions(model=bald_model, loader=pool_loader, device=self.device)

        # Evaluate BALD scores
        candidate_batch = get_bald_batch(
            pool_log_probs_N_K_C, batch_size=self.acquisition_size, dtype=torch.double, device=self.device
        )
        return candidate_batch

    def run(self, results):
        results["hparams"] = dataclasses.asdict(self)

        # Active Learning setup
        active_learning_data, validation_dataset, test_dataset = self.load_dataset()

        initial_training_set_indices = active_learning_data.get_random_pool_indices(self.initial_set_size)
        active_learning_data.acquire(initial_training_set_indices)

        results["initial_training_set_indices"] = initial_training_set_indices

        train_loader = torch.utils.data.DataLoader(
            active_learning_data.training_dataset,
            batch_size=64,
            sampler=RandomFixedLengthSampler(active_learning_data.training_dataset, self.samples_per_epoch),
        )
        pool_loader = torch.utils.data.DataLoader(active_learning_data.pool_dataset, batch_size=64, drop_last=False)

        validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64, drop_last=False)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, drop_last=False)

        results["active_learning_steps"] = []
        active_learning_steps = results["active_learning_steps"]

        # Active Training Loop
        while True:
            training_set_size = len(active_learning_data.training_dataset)
            print(f"Training set size {training_set_size}:")

            iteration_log = dict(training_log=[], evalution_metrics=None, acquisition=None)

            model = self.new_model()
            optimizer = self.new_optimizer(model)
            train(
                model=model,
                optimizer=optimizer,
                training_samples=self.num_training_samples,
                validation_samples=self.num_eval_samples,
                train_loader=train_loader,
                validation_loader=validation_loader,
                patience=self.num_patience_epochs,
                max_epochs=self.max_training_epochs,
                device=self.device,
                epochs_log=iteration_log["training_log"],
            )

            evaluation_metrics = evaluate(
                model=model, num_samples=self.num_eval_samples, loader=test_loader, device=self.device
            )
            iteration_log["evalution_metrics"] = evaluation_metrics
            print(f"Perf after training {evaluation_metrics}")

            if training_set_size >= self.max_training_set:
                print("Done.")
                break

            candidate_batch = self.get_candidate_batch(model, pool_loader)

            candidate_global_indices = get_base_indices(active_learning_data.pool_dataset, candidate_batch.indices)
            candidate_labels = [
                active_learning_data.dataset.targets[index].item() for index in candidate_global_indices
            ]

            iteration_log["acquisition"] = dict(
                indices=candidate_global_indices, labels=candidate_labels, scores=candidate_batch.scores
            )

            active_learning_data.acquire(candidate_batch.indices)

            ls = ", ".join(f"{label} ({score:.4})" for label, score in zip(candidate_labels, candidate_batch.scores))
            print(f"Acquiring (label, score)s: {ls}")

            active_learning_steps.append(iteration_log)

        return active_learning_steps

In [ ]:
# experiment

experiment = Experiment(max_training_epochs=1, max_training_set=150)

results = {}
experiment.run(results)

results